In [2]:
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files

# VEST Florida Validation

## Documentation

### Races

G16PRERTru - Donald J. Trump (Republican Party)  
G16PREDCli - Hillary Clinton (Democratic Party)  
G16PRELJoh - Gary Johnson (Libertarian Party)  
G16PRECCas - Darrell L. Castle (Constitution Party)  
G16PREGSte - Jill Stein (Green Party)  
G16PREIDeL - Roque De La Fuente (Reform Party)  
G16PREOth - Write-in Votes  

G16USSRRub - Marco Rubio (Republican Party)  
G16USSDMur - Patrick Murphy (Democratic Party)  
G16USSLSta - Paul Stanton (Libertarian Party)  
G16USSOth - Independent and Write-in Votes  

### Election Source

Election results from Florida Division of Elections (http://dos.myflorida.com/elections/data-statistics/elections-data/precinct-level-election-results/)  

### Precinct Source

The starting point was the Orlando Sentinel's precinct map for the 2016 presidential primary (http://interactive.orlandosentinel.com/elections/2016/presidential-primary/results/dem.html).  

Hillsborough, Lake, Miami-Dade, Orange, Palm Beach, and Pinellas Counties were updated directly with shapefiles downloaded from county Supervisor of Elections websites.  

Brevard, Marion, and Putnam Counties were updated with KML files from the respective county Supervisor of Elections websites, from their election result map pages.  

Columbia, DeSoto, Leon, Polk, and Osceola Counties updates received from Supervisor of Elections websites through personal contact.  

Highlands, Santa Rosa, Sarasota, and Volusia Counties had some precinct mergers, which were determined based on visual inspection of PDF precinct maps from the county websites.  

Charlotte, Hendry, Holmes, Jackson, Jefferson, Levy, Madison, and Walton Counties are from VTDs released by the Census Bureau's Redistricting Data Program.  

Hernando 99, Collier 450, and Palm Beach 8001/8002 are UOCAVA precincts with no real geography and votes were not redistributed.  

Precinct names were adjusted to align with the formatting used in the voter registration file (e.g., some precincts have leading spaces).  

In [8]:
#This Data is from the FL department of state, and can only be downloaded county by county
all_files = os.listdir("./raw-from-source/Election_Results/precinctlevelelectionresults2016gen")

In [9]:
#Check that all files have the same number of columns
for i in all_files:
    ref = "./raw-from-source/Election_Results/precinctlevelelectionresults2016gen/"
    file_ref = ref+i
    print(i)
    file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None, error_bad_lines=False)
    print(file_prev.shape)
    
#All the files have 19 columns, so they should be good to combine

SEM_PctResults20161108.txt
(8154, 19)
STJ_PctResults20161108.txt
(4276, 19)
BAK_PctResults20161108.txt
(909, 19)
SAN_PctResults20161108.txt
(3415, 19)
MRN_PctResults20161108.txt
(12201, 19)
DIX_PctResults20161108.txt
(858, 19)
JEF_PctResults20161108.txt
(1206, 19)
PAS_PctResults20161108.txt
(13784, 19)
DES_PctResults20161108.txt
(1414, 19)
TAY_PctResults20161108.txt
(1111, 19)
GIL_PctResults20161108.txt
(709, 19)
SUW_PctResults20161108.txt
(1556, 19)
GAD_PctResults20161108.txt


Skipping line 28: '	' expected after '"'
Skipping line 106: '	' expected after '"'
Skipping line 185: '	' expected after '"'
Skipping line 263: '	' expected after '"'
Skipping line 346: '	' expected after '"'
Skipping line 429: '	' expected after '"'
Skipping line 510: '	' expected after '"'
Skipping line 587: '	' expected after '"'
Skipping line 662: '	' expected after '"'
Skipping line 741: '	' expected after '"'
Skipping line 829: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 994: '	' expected after '"'
Skipping line 1076: '	' expected after '"'


(2584, 19)
HER_PctResults20161108.txt
(3037, 19)
MON_PctResults20161108.txt
(3168, 19)
WAK_PctResults20161108.txt
(929, 19)
JAC_PctResults20161108.txt
(1230, 19)
OSC_PctResults20161108.txt
(9550, 19)
POL_PctResults20161108.txt
(21010, 19)
WAL_PctResults20161108.txt
(1722, 19)
HOL_PctResults20161108.txt
(898, 19)
HAR_PctResults20161108.txt
(1392, 19)
STL_PctResults20161108.txt
(6331, 19)
LEV_PctResults20161108.txt
(1001, 19)
CAL_PctResults20161108.txt
(898, 19)
ALA_PctResults20161108.txt
(6478, 19)
VOL_PctResults20161108.txt
(11871, 19)
DAD_PctResults20161108.txt
(69874, 19)
HIL_PctResults20161108.txt
(45890, 19)
CHA_PctResults20161108.txt
(7195, 19)
WAS_PctResults20161108.txt
(1664, 19)
ESC_PctResults20161108.txt
(8067, 19)
ORA_PctResults20161108.txt
(32989, 19)
CLA_PctResults20161108.txt
(4275, 19)
GLA_PctResults20161108.txt
(1044, 19)
HAM_PctResults20161108.txt
(755, 19)
LIB_PctResults20161108.txt
(856, 19)
SUM_PctResults20161108.txt
(2379, 19)
HEN_PctResults20161108.txt
(1158, 19)
C

Skipping line 70: '	' expected after '"'
Skipping line 173: '	' expected after '"'
Skipping line 330: '	' expected after '"'
Skipping line 431: '	' expected after '"'
Skipping line 518: '	' expected after '"'
Skipping line 617: '	' expected after '"'
Skipping line 708: '	' expected after '"'
Skipping line 814: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 1014: '	' expected after '"'
Skipping line 1111: '	' expected after '"'
Skipping line 1210: '	' expected after '"'
Skipping line 1317: '	' expected after '"'
Skipping line 1421: '	' expected after '"'
Skipping line 1523: '	' expected after '"'
Skipping line 1628: '	' expected after '"'
Skipping line 1736: '	' expected after '"'
Skipping line 1831: '	' expected after '"'
Skipping line 1933: '	' expected after '"'
Skipping line 2025: '	' expected after '"'
Skipping line 2121: '	' expected after '"'
Skipping line 2218: '	' expected after '"'
Skipping line 2318: '	' expected after '"'
Skipping line 2422: '

Skipping line 24249: '	' expected after '"'
Skipping line 24344: '	' expected after '"'
Skipping line 24426: '	' expected after '"'
Skipping line 24520: '	' expected after '"'
Skipping line 24570: '	' expected after '"'
Skipping line 24634: '	' expected after '"'
Skipping line 24725: '	' expected after '"'
Skipping line 24825: '	' expected after '"'
Skipping line 24903: '	' expected after '"'
Skipping line 24972: '	' expected after '"'
Skipping line 25063: '	' expected after '"'
Skipping line 25157: '	' expected after '"'
Skipping line 25249: '	' expected after '"'
Skipping line 25334: '	' expected after '"'
Skipping line 25403: '	' expected after '"'
Skipping line 25485: '	' expected after '"'
Skipping line 25574: '	' expected after '"'
Skipping line 25666: '	' expected after '"'
Skipping line 25759: '	' expected after '"'
Skipping line 25854: '	' expected after '"'
Skipping line 25936: '	' expected after '"'
Skipping line 26026: '	' expected after '"'
Skipping line 26089: '	' expecte

Skipping line 46504: '	' expected after '"'
Skipping line 46602: '	' expected after '"'
Skipping line 46696: '	' expected after '"'
Skipping line 46790: '	' expected after '"'
Skipping line 46885: '	' expected after '"'
Skipping line 46971: '	' expected after '"'
Skipping line 47065: '	' expected after '"'
Skipping line 47163: '	' expected after '"'
Skipping line 47243: '	' expected after '"'
Skipping line 47328: '	' expected after '"'
Skipping line 47422: '	' expected after '"'
Skipping line 47516: '	' expected after '"'
Skipping line 47613: '	' expected after '"'
Skipping line 47714: '	' expected after '"'
Skipping line 47805: '	' expected after '"'
Skipping line 47903: '	' expected after '"'
Skipping line 48002: '	' expected after '"'
Skipping line 48101: '	' expected after '"'
Skipping line 48201: '	' expected after '"'
Skipping line 48295: '	' expected after '"'
Skipping line 48388: '	' expected after '"'
Skipping line 48471: '	' expected after '"'
Skipping line 48623: '	' expecte

Skipping line 67595: '	' expected after '"'
Skipping line 67674: '	' expected after '"'
Skipping line 67793: '	' expected after '"'
Skipping line 67794: '	' expected after '"'
Skipping line 67912: '	' expected after '"'
Skipping line 68009: '	' expected after '"'
Skipping line 68096: '	' expected after '"'
Skipping line 68180: '	' expected after '"'
Skipping line 68266: '	' expected after '"'
Skipping line 68357: '	' expected after '"'
Skipping line 68442: '	' expected after '"'
Skipping line 68526: '	' expected after '"'
Skipping line 68616: '	' expected after '"'
Skipping line 68707: '	' expected after '"'
Skipping line 68800: '	' expected after '"'
Skipping line 68883: '	' expected after '"'
Skipping line 68972: '	' expected after '"'
Skipping line 69049: '	' expected after '"'
Skipping line 69138: '	' expected after '"'
Skipping line 69233: '	' expected after '"'
Skipping line 69328: '	' expected after '"'
Skipping line 69390: '	' expected after '"'
Skipping line 69467: '	' expecte

(69962, 19)
OKA_PctResults20161108.txt
(4488, 19)
SAR_PctResults20161108.txt
(15496, 19)
CIT_PctResults20161108.txt
(2802, 19)
BRE_PctResults20161108.txt
(15505, 19)
BRO_PctResults20161108.txt
(69707, 19)
PIN_PctResults20161108.txt
(33620, 19)
MAD_PctResults20161108.txt
(1060, 19)
GUL_PctResults20161108.txt
(792, 19)
HIG_PctResults20161108.txt
(2856, 19)
MAN_PctResults20161108.txt
(7783, 19)
IND_PctResults20161108.txt
(6549, 19)


In [7]:
tay_values = pd.read_csv("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/pdv-fl/vest-fl-2016/raw-from-source/Election_Results/precinctlevelelectionresults2016gen/TAY_PctResults20161108.txt")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 76, saw 4
